In [153]:
#https://larevueia.fr/machine-learning-pour-la-classification-automatique-de-musiques-avec-python/

In [154]:
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import os 
import librosa

Préparation du calcul des features

In [160]:
zcr = [] #zero crossing rate

mean_spectral_centroids = [] # moyenne du Spectral centroid

rolloff_point = [] #calcul du spectral rolloff point

mfcc = [ [] for _ in range(20) ] #liste qui contiendra les listes de mfcc (mfcc1, mfcc2,...)



In [162]:
%%capture

# assign directory for the dataset
directory = 'Dataset'
 
# iterate over files in that directory

for filename in os.listdir(directory):
    sound_file = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(sound_file):
        #print(sound)
        audio = librosa.load(sound_file)[0]
        # Calcul du ZCR
        zcr0 = librosa.zero_crossings(audio)
        zcr.append(sum(zcr0))

        # Calcul de la moyenne du Spectral centroid
        spectral_centroids = librosa.feature.spectral_centroid(audio)[0]
        mean_spectral_centroids.append(np.mean(spectral_centroids))

        # Calcul du spectral rolloff point

        rolloff = librosa.feature.spectral_rolloff(audio)
        rolloff_point.append(np.mean(rolloff))

        # Calcul des moyennes des MFCC

        mfcc0 = librosa.feature.mfcc(audio) #liste de 20 liste (mfcc1, mfcc2, ...)

        for i in range(len(mfcc0)):
            mfcc[i].append(np.mean(mfcc0[i]))

        

In [167]:
filesnames = []

for filename in os.listdir(directory):
    sound_file = os.path.join(directory, filename)
    if os.path.isfile(sound_file):
        filesnames.append(sound_file)

In [168]:
print(len(zcr))

7017


In [169]:
n = len(filesnames)

assert len(zcr) == n
assert len(rolloff_point) == n
assert len(mean_spectral_centroids) == n

for i in range(len(mfcc)):
    assert len(mfcc[i]) == n



In [170]:
dico1 = {'Filesnames' : filesnames,
'zcr': zcr,
'spectral_c':mean_spectral_centroids,
'rolloff': rolloff_point
}
dico2 = {
    'mfcc' + str(i): mfcc[i-1] for i in range(1,21)
}

In [171]:
data_features_dico = dico1 | dico2

In [175]:
data_features_df = DataFrame(data_features_dico, columns= ['Filesnames', 'zcr', 'spectral_c', 'rolloff', 'mfcc1', 'mfcc2', 'mfcc3',
                'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9',
                'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15',
                'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19', 'mfcc20'])


In [176]:
export_csv_features = data_features_df.to_csv ('data_features.csv', index = None, header=True, encoding='utf-8')      #, sep=';')

In [178]:
data_features_df


,Filesnames,zcr,spectral_c,rolloff,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20
0,Dataset\Autre100263-2-0-117.wav,12323,2111.629331,3967.399324,-420.238251,103.904526,-47.550430,55.606483,5.350782,12.434221,6.779348,6.292046,2.729948,8.500508,-7.737784,17.959957,-6.742780,11.154058,1.755470,5.540475,-1.832767,10.480143,-5.762805,6.669657
1,Dataset\Autre100263-2-0-121.wav,9059,1818.612681,3664.378613,-455.256805,116.408104,-41.610699,47.207104,4.650426,19.322447,12.985572,5.075196,3.053262,7.610181,-8.801887,11.849342,-1.554925,10.021297,0.559735,6.055298,-6.639349,3.747846,-2.924167,12.053761
2,Dataset\Autre100263-2-0-126.wav,11541,2244.620136,4423.641844,-409.939392,96.085541,-29.919291,47.638351,6.763555,9.838276,7.694974,3.081153,2.038530,5.989011,-9.093197,18.593916,-3.856829,11.185817,-3.625854,3.480807,-0.527970,7.954938,-1.339017,6.880064
3,Dataset\Autre100263-2-0-137.wav,10477,1981.227732,3907.591786,-442.846558,108.394539,-47.178040,55.187874,7.742001,20.302290,7.347321,5.860622,7.027884,6.742701,-11.407565,14.406661,-1.839403,8.542976,2.805647,9.005601,-5.710881,5.383008,-7.941531,12.409350
4,Dataset\Autre100263-2-0-143.wav,6499,1767.169485,3765.136578,-443.650665,112.543655,-29.388401,51.108513,6.998570,21.019321,9.774244,7.921374,2.132150,7.051430,-5.272412,11.670588,0.060888,9.338826,0.210148,3.348249,-5.759408,5.665868,-4.361087,8.804883
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7012,Dataset\Feu4_9_5.wav,11685,3676.382361,8217.904257,-85.856033,44.797367,54.510426,32.032497,13.352752,30.406040,7.842577,18.289858,-3.253780,11.901505,-5.270272,8.899734,-6.188625,7.078750,-7.201250,6.310044,-6.838622,6.171902,-7.197029,4.589937
7013,Dataset\Feu4_9_6.wav,14762,4032.305875,8380.834679,-76.643387,40.962029,53.552555,30.839485,12.678088,29.697887,6.531869,16.938023,-4.792107,10.967959,-6.036715,8.236232,-7.537588,5.695540,-8.237390,4.225119,-8.250767,5.240402,-7.863617,3.571088
7014,Dataset\Feu4_9_7.wav,14861,3996.022940,8362.350975,-72.589615,41.976276,54.391838,29.659508,12.278446,29.870834,6.320203,17.175236,-4.743874,11.610956,-6.362481,7.842653,-7.305359,5.683318,-8.195653,5.130675,-7.981054,5.436183,-6.956571,3.947755
7015,Dataset\Feu4_9_8.wav,11593,3705.594813,8223.443145,-75.076286,43.856670,56.520439,32.219421,12.664134,30.471985,7.113659,17.708168,-3.060562,12.178458,-5.381196,9.123774,-6.520344,6.442585,-6.962236,5.517067,-6.693164,6.679021,-6.807524,4.914062


On va maintenant rajouter le label 1 pour les feu et -1 sinon

In [183]:
labels = []

for audio in filesnames:
    if audio[8:13] == "Autre":
        labels.append(-1)
    else:
        labels.append(1)

In [188]:
data_features_df_labels = data_features_df.assign(Labels = labels)



In [189]:
data_features_df_labels

,Filesnames,zcr,spectral_c,rolloff,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,Labels
0,Dataset\Autre100263-2-0-117.wav,12323,2111.629331,3967.399324,-420.238251,103.904526,-47.550430,55.606483,5.350782,12.434221,6.779348,6.292046,2.729948,8.500508,-7.737784,17.959957,-6.742780,11.154058,1.755470,5.540475,-1.832767,10.480143,-5.762805,6.669657,-1
1,Dataset\Autre100263-2-0-121.wav,9059,1818.612681,3664.378613,-455.256805,116.408104,-41.610699,47.207104,4.650426,19.322447,12.985572,5.075196,3.053262,7.610181,-8.801887,11.849342,-1.554925,10.021297,0.559735,6.055298,-6.639349,3.747846,-2.924167,12.053761,-1
2,Dataset\Autre100263-2-0-126.wav,11541,2244.620136,4423.641844,-409.939392,96.085541,-29.919291,47.638351,6.763555,9.838276,7.694974,3.081153,2.038530,5.989011,-9.093197,18.593916,-3.856829,11.185817,-3.625854,3.480807,-0.527970,7.954938,-1.339017,6.880064,-1
3,Dataset\Autre100263-2-0-137.wav,10477,1981.227732,3907.591786,-442.846558,108.394539,-47.178040,55.187874,7.742001,20.302290,7.347321,5.860622,7.027884,6.742701,-11.407565,14.406661,-1.839403,8.542976,2.805647,9.005601,-5.710881,5.383008,-7.941531,12.409350,-1
4,Dataset\Autre100263-2-0-143.wav,6499,1767.169485,3765.136578,-443.650665,112.543655,-29.388401,51.108513,6.998570,21.019321,9.774244,7.921374,2.132150,7.051430,-5.272412,11.670588,0.060888,9.338826,0.210148,3.348249,-5.759408,5.665868,-4.361087,8.804883,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7012,Dataset\Feu4_9_5.wav,11685,3676.382361,8217.904257,-85.856033,44.797367,54.510426,32.032497,13.352752,30.406040,7.842577,18.289858,-3.253780,11.901505,-5.270272,8.899734,-6.188625,7.078750,-7.201250,6.310044,-6.838622,6.171902,-7.197029,4.589937,1
7013,Dataset\Feu4_9_6.wav,14762,4032.305875,8380.834679,-76.643387,40.962029,53.552555,30.839485,12.678088,29.697887,6.531869,16.938023,-4.792107,10.967959,-6.036715,8.236232,-7.537588,5.695540,-8.237390,4.225119,-8.250767,5.240402,-7.863617,3.571088,1
7014,Dataset\Feu4_9_7.wav,14861,3996.022940,8362.350975,-72.589615,41.976276,54.391838,29.659508,12.278446,29.870834,6.320203,17.175236,-4.743874,11.610956,-6.362481,7.842653,-7.305359,5.683318,-8.195653,5.130675,-7.981054,5.436183,-6.956571,3.947755,1
7015,Dataset\Feu4_9_8.wav,11593,3705.594813,8223.443145,-75.076286,43.856670,56.520439,32.219421,12.664134,30.471985,7.113659,17.708168,-3.060562,12.178458,-5.381196,9.123774,-6.520344,6.442585,-6.962236,5.517067,-6.693164,6.679021,-6.807524,4.914062,1


In [190]:
#enregistrement
export_csv_features_labels = data_features_df_labels.to_csv ('data_features_labels.csv', index = None, header=True, encoding='utf-8')  